In [1]:
!pip install pycaret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.7/484.7 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.4/153.4 kB 17.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 85.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 74.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [20]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score
from pycaret.regression import *

SEED = 12345
N_SPLIT = 5

In [3]:
from google.colab import drive
drive.mount("/content/drive/")
INPUT_PATH_1 = "/content/drive/My Drive/HousePrices/input/"
INPUT_PATH_2 = "/content/drive/My Drive/HousePrices/features/"
OUTPUT_PATH = "/content/drive/My Drive/HousePrices/prediction/"

Mounted at /content/drive/


In [12]:
df = pd.read_csv(INPUT_PATH_2 + "data_v0.csv")
print(df.shape)
df.head(3)

(2919, 82)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,train_test
0,784,85,3.0,NaN,9101,1.0,NaN,0.0,3.0,0.0,...,NaN,NaN,NaN,0,7,2009,8.0,4.0,165500.0,1
1,512,120,3.0,40.0,6792,1.0,NaN,0.0,3.0,0.0,...,NaN,NaN,NaN,0,3,2006,6.0,5.0,202665.0,1
2,1072,60,3.0,78.0,11700,1.0,NaN,3.0,3.0,0.0,...,NaN,NaN,NaN,0,5,2009,8.0,4.0,154000.0,1


In [13]:
df_train = df.loc[(df['train_test'].isin([1,2])), :].drop(columns=['train_test','Id'], axis=1)
#y_train = df.loc[(df['train_test'].isin([1,2])), :]['SalePrice']
#print(x_train.shape)
#print(y_train.shape)
df_train.head(3)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,85,3.0,NaN,9101,1.0,NaN,0.0,3.0,0.0,0.0,...,0,NaN,NaN,NaN,0,7,2009,8.0,4.0,165500.0
1,120,3.0,40.0,6792,1.0,NaN,0.0,3.0,0.0,4.0,...,0,NaN,NaN,NaN,0,3,2006,6.0,5.0,202665.0
2,60,3.0,78.0,11700,1.0,NaN,3.0,3.0,0.0,4.0,...,0,NaN,NaN,NaN,0,5,2009,8.0,4.0,154000.0


In [21]:
# 前処理
cat_list = ['Neighborhood','BsmtFinType1','BsmtFinType2']
tmp = setup(data=df_train, target='SalePrice', categorical_features=cat_list, fold=N_SPLIT, session_id=SEED)

,Description,Value
0,Session id,12345
1,Target,SalePrice
2,Target type,Regression
3,Original data shape,"(1460, 80)"
4,Transformed data shape,"(1460, 114)"
5,Transformed train set shape,"(1021, 114)"
6,Transformed test set shape,"(439, 114)"
7,Numeric features,76
8,Categorical features,3
9,Rows with missing values,100.0%


In [22]:
# 分析手法の比較
compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,17510.3951,909076709.4269,29316.8272,0.8652,0.1398,0.0998,0.4340
et,Extra Trees Regressor,17856.4819,926306854.3451,30032.1427,0.8630,0.1456,0.1039,2.0320
gbr,Gradient Boosting Regressor,17352.5345,923306484.3427,29609.5525,0.8617,0.1386,0.0995,0.7040
rf,Random Forest Regressor,18547.1980,1020351604.1847,31278.9434,0.8476,0.1499,0.1074,1.6460
xgboost,Extreme Gradient Boosting,19364.6547,1062838899.2000,32228.6578,0.8384,0.1580,0.1102,1.3440
ada,AdaBoost Regressor,24722.3908,1361478662.9951,36524.8343,0.7932,0.1962,0.1570,0.6060
en,Elastic Net,20622.3325,1517984945.2878,37004.2642,0.7699,0.1712,0.1182,0.2840
ridge,Ridge Regression,21108.4915,1651118459.2835,38880.7213,0.7496,0.1689,0.1235,0.1500
llar,Lasso Least Angle Regression,22572.5486,1650856838.0379,38746.2511,0.7492,0.1874,0.1313,0.1760
dt,Decision Tree Regressor,27459.2564,1806916048.1654,42167.7474,0.7237,0.2176,0.1596,0.1480


Processing:   0%|          | 0/81 [00:00<?, ?it/s]

LGBMRegressor(random_state=12345)

In [25]:
# 検証
model = create_model('et') # lightgbm

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,14017.6092,434523404.8483,20845.2250,0.9130,0.1136,0.0831
1,18267.3794,981681558.2440,31331.7979,0.8633,0.1533,0.1082
2,19911.7659,923530085.2791,30389.6378,0.8674,0.1508,0.1126
3,18446.5717,1016573809.9424,31883.7546,0.8720,0.1484,0.1022
4,18639.0831,1275225413.4115,35710.2984,0.7992,0.1617,0.1135
Mean,17856.4819,926306854.3451,30032.1427,0.8630,0.1456,0.1039
Std,2004.5270,273794562.9888,4937.3329,0.0365,0.0166,0.0112


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
#print(model)

In [ ]:
tuned_model = tune_model(model, optimize='rmse', fold=N_SPLIT, n_iter=30, verbose=True)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,15849.4340,544018042.1065,23324.1944,0.8911,0.1202,0.0922
1,19317.3785,1095466723.6265,33097.8356,0.8475,0.1591,0.1133
2,19964.5002,842461399.4892,29025.1856,0.8790,0.1548,0.1164
3,19342.9963,1122392335.3395,33502.1243,0.8587,0.1525,0.1082
4,19319.6871,1266355425.4493,35585.8880,0.8006,0.1630,0.1167
Mean,18758.7992,974138785.2022,30907.0456,0.8554,0.1499,0.1094
Std,1475.5347,254728457.8359,4346.6445,0.0313,0.0153,0.0091


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 5 folds for each of 30 candidates, totalling 150 fits


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [ ]:
# evaluate_model(model)

In [ ]:
# モデルの確定（データ全体で学習）
fix_model = finalize_model(tuned_model)
display(fix_model)

In [22]:
# submission
df_sub = df.loc[(df['train_test']==3), :].drop(columns=['train_test','Id','SalePrice'], axis=1)
print(df_sub.shape)
df_sub.head(3)

In [ ]:
df_out = df.loc[(df['train_test']==3), :][['PassengerId']]
df_out.head(3)

In [ ]:
# 予測
df_pred_sub = predict_model(model)
df_pred_sub = predict_model(fix_model, data=df_sub)
#print(df_pred_sub['prediction_label'].value_counts()/len(df_pred_sub))
#print(df_pred_sub['prediction_score'].mean())
#df_pred_sub.head(3)

# df_sub = df_sub[['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']]
# print(df_sub.shape)
# df_sub.head(3)

In [ ]:
df_out['Survived'] = df_pred_sub['prediction_label']
df_out.to_csv(OUTPUT_PATH_1 + "submission_pycaret_cv_v1.csv", index=False)
df_out.head(3)